### Task evaluation

#### Loading Models
Please note the models output `task,domain,feature`

In [1]:
!git clone https://github.com/scaairesearch/da_cv_fer.git
import os
os.getcwd()
os.chdir('da_cv_fer')

Cloning into 'da_cv_fer'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 87 (delta 37), reused 79 (delta 35), pack-reused 0
Receiving objects: 100% (87/87), 2.94 MiB | 13.60 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [2]:
!pip install -q -r requirements.txt # -q is for quiet install
!pip install -q opendatasets # quient install, for some reason this does not get installed with requirements

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.7/301.7 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.7/225.7 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.2/154.2 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.0/807.0 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [3]:
import torch
from pathlib import Path
import pandas as pd

import torch.nn as nn

from da_cv_fer.data_config import DataConfig
from da_cv_fer.ds_sfew import DatasetSFEW
from da_cv_fer.ds_expw import EXPW
from da_cv_fer.utils import show_batch
from da_cv_fer.model_DANN import DANN
from da_cv_fer.run_config import RunConfig
from da_cv_fer.train import (weights_init,
                             train_model,
                             get_correct_predictions,
                             test_model)

In [4]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


#### Loading Models to evaluate

In [10]:
dataconfig_inference = DataConfig()

In [11]:
non_dann_model_sfew_name = 'non_dann_sfew_20_01.pt'
dann_model_sfew_expw_name = 'dann_sfew_expw_20_01.pt'
ewc_dann_model_sfew_expw_name = 'ewc_dann_sfew_expw_20_01.pt'

In [12]:
# NON DANN
non_dann_model_inference = DANN(num_classes=7) # skeleton copy
NON_DANN_SFEW_DIR = dataconfig_inference.NON_DANN_SFEW_DIR # Path(self.MODEL_DIR,"non_dann_sfew")

try:
    non_dann_model_inference.load_state_dict(torch.load(Path(NON_DANN_SFEW_DIR, non_dann_model_sfew_name),
                                                        map_location=torch.device(device)))
    print(f'{non_dann_model_sfew_name} locked and loaded')
except:
    print(f' problem in loading {non_dann_model_sfew_name}')

# DANN
dann_model_sfew_expw_inference = DANN(num_classes=7) # skeleton copy
DANN_SFEW_EXPW_DIR = dataconfig_inference.DANN_SFEW_EXPW_DIR # Path(self.MODEL_DIR,"dann_sfew_expw")
try:
    dann_model_sfew_expw_inference.load_state_dict(torch.load(Path(DANN_SFEW_EXPW_DIR, dann_model_sfew_expw_name),
                                                              map_location=torch.device(device)))
    print(f'{dann_model_sfew_expw_name} locked and loaded')
except:
    print(f' problem in loading {dann_model_sfew_expw_name}')

# EWC DANN
ewc_dann_model_sfew_expw_inference = DANN(num_classes=7) # skeleton copy
EWC_DANN_SFEW_EXPW_DIR = dataconfig_inference.EWC_DANN_SFEW_EXPW_DIR # Path(self.MODEL_DIR,"ewc_dann_sfew_expw")
try:
    ewc_dann_model_sfew_expw_inference.load_state_dict(torch.load(Path(EWC_DANN_SFEW_EXPW_DIR, ewc_dann_model_sfew_expw_name),
                                                              map_location=torch.device(device)))
    print(f'{ewc_dann_model_sfew_expw_name} locked and loaded')
except:
    print(f' problem in loading {ewc_dann_model_sfew_expw_name}')


non_dann_sfew_20_01.pt locked and loaded
dann_sfew_expw_20_01.pt locked and loaded
ewc_dann_sfew_expw_20_01.pt locked and loaded


### Dataloaders for source and target

In [6]:
# Mounting Google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
sfew = DatasetSFEW()
sfew_train_loader, sfew_val_loader = sfew.get_dataloader()


self.BASE_PATH -dataset,
 self.EXTRACT_DIR-dataset/sfew,
 self.ZIP_FILE_PATH - dataset/SFEW_2.zip 
Directory dataset/sfew created successfully.
No files (including zip file) found in dataset/sfew.Copying file
Starting File copying from /content/gdrive/MyDrive/CV_FER/dataset/SFEW_2.zip to dataset/sfew
File copied successfully from /content/gdrive/MyDrive/CV_FER/dataset/SFEW_2.zip to dataset/sfew
Initiating extraction of dataset/sfew/SFEW_2.zip to dataset/sfew
...completed for dataset/sfew/SFEW_2.zip
dataset/sfew/Train Neutral.zip Neutral
...completed for dataset/sfew/Train/Neutral.zip
dataset/sfew/Train Sad.zip Sad
...completed for dataset/sfew/Train/Sad.zip
dataset/sfew/Train Happy.zip Happy
...completed for dataset/sfew/Train/Happy.zip
dataset/sfew/Train Angry.zip Angry
...completed for dataset/sfew/Train/Angry.zip
dataset/sfew/Train Disgust.zip Disgust
...completed for dataset/sfew/Train/Disgust.zip
dataset/sfew/Train Fear.zip Fear
...completed for dataset/sfew/Train/Fear.zip
dataset

In [8]:
expw_obj = EXPW()
expw_train_ds, expw_val_ds = expw_obj.get_dataset()
expw_train_loader, expw_val_loader = expw_obj.get_dataloader()

desitination file path = dataset/expwds/expwds.zip
in EXPW()...
Directory dataset/expwds created successfully.


100%|██████████| 7.68G/7.68G [01:23<00:00, 98.5MB/s]



File downloaded successfully from https://www.kaggle.com/datasets/mohammedaaltaha/expwds to dataset/expwds/expwds.zip
Unzipped Files already exist in dataset/expwds, not extracted
Directory dataset/expwds already exists.
Directory dataset/expwds already exists.


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


#### Printing comparison

In [13]:
#@title Typical Test Function
#TODO : move this to common file

def get_correct_predictions(prediction, labels):
    """
    Function to return total number of correct predictions
    :param prediction: Model predictions on a given sample of data
    :param labels: Correct labels of a given sample of data
    :return: Number of correct predictions
    """
    return prediction.argmax(dim=1).eq(labels.argmax(dim=1)).sum().item()

def test_model(model, device, data_loader, mode= 'test',
               class_loss_function = nn.CrossEntropyLoss() ):
    model.eval() # evaluation mode

    # variables to track loss and accuracy
    test_loss = 0.0
    correct = 0

    # Disable gradient updation
    with torch.no_grad():
        # Iterate over each batch and fetch images and labels from the batch
        for batch_idx, batch in enumerate(data_loader):
            images, labels = batch[0].to(device), batch[1].to(device)

            class_output, _ , _ = model(images) # forward pass, output = {class, domain, feature}
            class_loss = class_loss_function(class_output,labels) # Calculate Loss

            test_loss += class_loss.item() # sum up batch loss

            # Sum up batch correct predictions
            correct += get_correct_predictions(class_output, labels)

    # Calculate test loss for a epoch
    test_loss /= len(data_loader.dataset)
    accuracy = 100. * correct / len(data_loader.dataset)

    print('Test / Val: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(test_loss, correct, len(data_loader.dataset),
        accuracy))


    return correct, test_loss, accuracy



In [ ]:
# Moving the models to device (GPU or CPU)
non_dann_model_inference = non_dann_model_inference.to(device)
dann_model_sfew_expw_inference = dann_model_sfew_expw_inference.to(device)
ewc_dann_model_sfew_expw_inference= ewc_dann_model_sfew_expw_inference.to(device)

In [15]:
runconfig = RunConfig()
_,_,non_dann_accuracy_source = test_model(model =non_dann_model_inference,
                                 device=device,
                                 data_loader=sfew_val_loader,
                                 class_loss_function=runconfig.criterion_class )

_,_,non_dann_accuracy_target = test_model(model =non_dann_model_inference,
                                 device=device,
                                 data_loader=expw_val_loader,
                                 class_loss_function=runconfig.criterion_class )

_,_,dann_accuracy_source = test_model(model =dann_model_sfew_expw_inference,
                                 device=device,
                                 data_loader=sfew_val_loader,
                                 class_loss_function=runconfig.criterion_class )

_,_,dann_accuracy_target = test_model(model =dann_model_sfew_expw_inference,
                                 device=device,
                                 data_loader=expw_val_loader,
                                 class_loss_function=runconfig.criterion_class )

_,_,ewc_dann_accuracy_source = test_model(model =ewc_dann_model_sfew_expw_inference,
                                 device=device,
                                 data_loader=sfew_val_loader,
                                 class_loss_function=runconfig.criterion_class )

_,_,ewc_dann_accuracy_target = test_model(model =ewc_dann_model_sfew_expw_inference,
                                 device=device,
                                 data_loader=expw_val_loader,
                                 class_loss_function=runconfig.criterion_class )


Test / Val: Average loss: 0.1684, Accuracy: 101/436 (23.17%)

Test / Val: Average loss: 0.1704, Accuracy: 2590/13620 (19.02%)

Test / Val: Average loss: 0.1962, Accuracy: 109/436 (25.00%)

Test / Val: Average loss: 0.2043, Accuracy: 2734/13620 (20.07%)

Test / Val: Average loss: 0.1950, Accuracy: 110/436 (25.23%)

Test / Val: Average loss: 0.2023, Accuracy: 2756/13620 (20.23%)



In [ ]:
# ewc_dann_model_sfew_expw_name = 'ewc_dann_sfew_expw_20_12.pt'
# ewc_dann_model_sfew_expw_inference = DANN(num_classes=7) # skeleton copy
# EWC_DANN_SFEW_EXPW_DIR = dataconfig_inference.EWC_DANN_SFEW_EXPW_DIR # Path(self.MODEL_DIR,"ewc_dann_sfew_expw")
# try:
#     ewc_dann_model_sfew_expw_inference.load_state_dict(torch.load(Path(EWC_DANN_SFEW_EXPW_DIR, ewc_dann_model_sfew_expw_name),
#                                                               map_location=torch.device(device)))
#     print(f'{ewc_dann_model_sfew_expw_name} locked and loaded')
# except:
#     print(f' problem in loading {ewc_dann_model_sfew_expw_name}')

# _,_,ewc_dann_accuracy_source = test_model(model =ewc_dann_model_sfew_expw_inference,
#                                  device=device,
#                                  data_loader=sfew_val_loader,
#                                  class_loss_function=runconfig.criterion_class )

# _,_,ewc_dann_accuracy_target = test_model(model =ewc_dann_model_sfew_expw_inference,
#                                  device=device,
#                                  data_loader=expw_val_loader,
#                                  class_loss_function=runconfig.criterion_class )

In [16]:
source= 'sfew'
target = 'expw'

col_name = ['model_class','trained_on','accuracy_source','accuracy_target']
df_nice_results = pd.DataFrame(columns = col_name)
print(df_nice_results)
dict_input = {}
dict_input = {'model_class': 'Non_DANN',
                   'trained_on': str(source)+ " source",
                #    'accuracy_source':dict_non_dann_losses_list_sfew_expw['test_accuracy_source'],
                #    'accuracy_target':dict_non_dann_losses_list_sfew_expw['test_accuracy_target'],
                   'accuracy_source':non_dann_accuracy_source,
                   'accuracy_target':non_dann_accuracy_target
                   }
print(dict_input)
df_nice_results = pd.concat([df_nice_results, pd.DataFrame(dict_input, index=[0])], ignore_index=True)


dict_input = {'model_class': 'DANN',
                   'trained_on': str(source)+" source + "+str(target) + " target",
                  #  'accuracy_source':dict_dann_losses_list_sfew_expw['test_accuracy_source'],
                  #  'accuracy_target':dict_dann_losses_list_sfew_expw['test_accuracy_target'],
                   'accuracy_source':dann_accuracy_source,
                   'accuracy_target':dann_accuracy_target
                   }
print(dict_input)

df_nice_results = pd.concat([df_nice_results, pd.DataFrame(dict_input, index=[0])], ignore_index=True)


dict_input = {'model_class': 'EWC_DANN',
                   'trained_on': str(source)+" source + "+str(target) + " target",
                  #  'accuracy_source':dict_ewc_dann_losses_list_sfew_expw['test_accuracy_source'],
                  #  'accuracy_target':dict_ewc_dann_losses_list_sfew_expw['test_accuracy_target'],
                   'accuracy_source':ewc_dann_accuracy_source,
                   'accuracy_target':ewc_dann_accuracy_target
                   }

df_nice_results = pd.concat([df_nice_results, pd.DataFrame(dict_input, index=[0])], ignore_index=True)


df_nice_results

Empty DataFrame
Columns: [model_class, trained_on, accuracy_source, accuracy_target]
Index: []
{'model_class': 'Non_DANN', 'trained_on': 'sfew source', 'accuracy_source': 23.165137614678898, 'accuracy_target': 19.016152716593247}
{'model_class': 'DANN', 'trained_on': 'sfew source + expw target', 'accuracy_source': 25.0, 'accuracy_target': 20.073421439060205}


,model_class,trained_on,accuracy_source,accuracy_target
0,Non_DANN,sfew source,23.165138,19.016153
1,DANN,sfew source + expw target,25.000000,20.073421
2,EWC_DANN,sfew source + expw target,25.229358,20.234949


### Bias Evaluation